In [ ]:
import pandas as pd
import datetime
import random
import uuid


users = pd.DataFrame(columns=['UserId', 'Name', 'Email', 'Password', 'Phone', 'MailingAddress', 'BillingAddress', 'PreferredDinerNumber', 'EarnedPoints', 'PreferredPaymentMethod'])
reservations = pd.DataFrame(columns=['UserId', 'Name', 'Phone', 'Email', 'DateTime', 'Guests', 'TableIds', 'PaymentMethod', 'MailingAddress', 'BillingAddress'])
tables = pd.DataFrame(columns=['TableId', 'Capacity'])

def register_user():
    user_id = len(users) + 1
    name = input("Enter your name: ")
    phone = input("Enter your phone number: ")
    email = input("Enter your email address: ")
    password = input("Enter your password: ")
    mailing_address = input("Enter your mailing address: ")
    billing_address = input("Enter your billing address (press enter if same as mailing address): ")
    if not billing_address:
        billing_address = mailing_address
    preferred_diner_number = len(users) + 1
    earned_points = 0
    preferred_payment_method = input("Enter your preferred payment method (cash, credit, check): ")
    new_user = {'UserId': user_id, 'Name': name, 'Email': email, 'Password': password, 'Phone': phone, 'MailingAddress': mailing_address, 'BillingAddress': billing_address, 'PreferredDinerNumber': preferred_diner_number, 'EarnedPoints': earned_points, 'PreferredPaymentMethod': preferred_payment_method}
    return new_user

def login():
    email = input("Enter your email address: ")
    password = input("Enter your password: ")
    user = users.loc[(users['Email'] == email) & (users['Password'] == password)]
    if not user.empty:
        return user.iloc[0]
    else:
        print("User not found or incorrect password.")
        return None

def is_high_traffic(date_time):
    # Customize the high-traffic hours and dates according to your needs
    high_traffic_hours = range(18, 22)
    high_traffic_dates = [datetime.date(year=date_time.year, month=7, day=4)]

    if date_time.date() in high_traffic_dates or (date_time.weekday() < 5 and date_time.hour in high_traffic_hours):
        return True
    return False

def get_payment_method(user):
    if user is not None:
        return user['PreferredPaymentMethod']
    return input("Enter your payment method (cash, credit, check): ")

def create_tables():
    capacities = [2, 4, 6, 8]
    num_tables = 10
    table_list = []
    for i in range(1, num_tables + 1):
        capacity = random.choice(capacities)
        table_id = str(uuid.uuid4())
        table_list.append({'TableId': table_id, 'Capacity': capacity})
    return pd.DataFrame(table_list)


tables = create_tables()


def reserve_tables(guests):
    def find_combinations(guests):
        combinations = []
        for i in range(1, guests):
            combinations.append((i, guests - i))
        return combinations
    
    available_tables = tables[tables['Capacity'] >= guests]
    if not available_tables.empty:
        table_id = available_tables.iloc[0]['TableId']
        tables.drop(available_tables.index[0], inplace=True)
        return [table_id]
    else:
        possible_combinations = find_combinations(guests)
        for combination in possible_combinations:
            available_tables1 = tables[tables['Capacity'] == combination[0]]
            available_tables2 = tables[tables['Capacity'] == combination[1]]
            if not available_tables1.empty and not available_tables2.empty:
                table_id1 = available_tables1.iloc[0]['TableId']
                table_id2 = available_tables2.iloc[0]['TableId']
                tables.drop(available_tables1.index[0], inplace=True)
                tables.drop(available_tables2.index[0], inplace=True)
                print("Tables combined to accommodate your request.")
                return [table_id1, table_id2]
        return None


def create_reservation(user, is_guest):
    if is_guest:
        name = input("Enter your name: ")
        phone = input("Enter your phone number: ")
        email = input("Enter your email address: ")
        mailing_address = None
        billing_address = None
    else:
        name = user['Name']
        phone = user['Phone']
        email = user['Email']
        mailing_address = user['MailingAddress']
        billing_address = user['BillingAddress']

    date = input("Enter the date (YYYY-MM-DD): ")
    time = input("Enter the time (HH:MM): ")

    date_time = datetime.datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    guests = int(input("Enter the number of guests: "))

    user_id = None if is_guest else user['UserId']

    if is_high_traffic(date_time):
        print("High-traffic hours detected. A valid payment method is required.")
        payment_method = get_payment_method(user)
    else:
        payment_method = None

    reserved_table_ids = reserve_tables(guests)

    if reserved_table_ids:
        reservation = {'UserId': user_id, 'Name': name, 'Phone': phone, 'Email': email, 'DateTime': date_time, 'Guests': guests, 'TableIds': reserved_table_ids, 'PaymentMethod': payment_method, 'MailingAddress': mailing_address, 'BillingAddress': billing_address}
        reservations.loc[len(reservations)] = reservation
        if is_guest:
            print("Consider registering to enjoy additional benefits.")
        print(f"Reservation successful. Your table(s): {', '.join(str(table_id) for table_id in reserved_table_ids)}")
        display_reservation_details(reservation)
    else:
        print("No tables available for your reservation.")



        
        
def display_reservation_details(reservation):
    print("\nReservation Details:")
    print(f"Name: {reservation['Name']}")
    print(f"Phone: {reservation['Phone']}")
    print(f"Email: {reservation['Email']}")
    print(f"Date and Time: {reservation['DateTime'].strftime('%Y-%m-%d %H:%M')}")
    print(f"Number of Guests: {reservation['Guests']}")
    print(f"Table(s): {', '.join(str(table_id) for table_id in reservation['TableIds'])}")
    if reservation['PaymentMethod']:
        print(f"Payment Method: {reservation['PaymentMethod']}")
    if reservation['MailingAddress']:
        print(f"Mailing Address: {reservation['MailingAddress']}")
    if reservation['BillingAddress']:
        print(f"Billing Address: {reservation['BillingAddress']}")



def main():
    global users, reservations, tables

    # Initialize tables
    for i in range(1, 11):
        tables.loc[len(tables)] = {'TableId': str(uuid.uuid4()), 'Capacity': 2 if i % 2 == 0 else 4}

    while True:
        print("\nOptions:")
        print("1. Register user")
        print("2. Login and reserve a table")
        print("3. Guest reservations")
        print("4. Show reservations details")
        print("5. Show table capacities")
        print("6. Exit")

        option = int(input("Select an option (1-6): "))

        if option == 1:
            new_user = register_user()
            users.loc[len(users)] = new_user
            print(f"User registered. Your Preferred Diner Number is {new_user['PreferredDinerNumber']}.")

        elif option == 2:
            user = login()
            if user is not None:
                create_reservation(user, False)

        elif option == 3:
            create_reservation(None, True)

        elif option == 4:
            print("Reservation details:")
            print(reservations)

        elif option == 5:
            print("Table capacities:")
            print(tables)

        elif option == 6:
            print("Exiting the program.")
            break

        else:
            print("Invalid option. Please select a valid option (1-6).")

if __name__ == "__main__":
    main()



Options:
1. Register user
2. Login and reserve a table
3. Guest reservations
4. Show reservations details
5. Show table capacities
6. Exit
Select an option (1-6): 1
